In [14]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

### RESNEXT_29_4x64D

#### Without Containers

In [3]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

  Cloning https://github.com/NVIDIA/apex.git to /tmp/pip-install-nyqnqino/apex
  Running command git clone -q https://github.com/NVIDIA/apex.git /tmp/pip-install-nyqnqino/apex
  Running command git submodule update --init --recursive -q
  Created wheel for fvcore: filename=fvcore-0.1.1.post20200623-py3-none-any.whl size=41179 sha256=29ea8fe5e36b186ccd4fdcae487cfd71708084adf0e9e08a3df0ed0dd745177f
  Stored in directory: /home/ec2-user/.cache/pip/wheels/27/fc/86/b29aea030f5468db673ec86033a9579cc50e02979aa0c78ebe
  Created wheel for apex: filename=apex-0.1-py3-none-any.whl size=192130 sha256=61d4ae3a9db07dda87ad2c3ee8d4ce64ba366c7625874f4121dde6344515ad8b
  Stored in directory: /tmp/pip-ephem-wheel-cache-8yyjymph/wheels/2a/45/23/6b4f2d6323a65ee0022d22a96d7bf580138e689f17cc48235c
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=3d31b3d3eab9309bcd258d4ea87ae67d714a3f50ec4a667a07b0c21399e75e17
  Stored in directory: /home/ec2-user/.cache/pip/wheels/93

In [5]:
# Need to add this to requirements.txt
!pip install tensorboard

     |████████████████████████████████| 3.0 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 27.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 50.2 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 52.5 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 49.7 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 48.3 MB/s eta 0:00:01
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121931 sha256=d0619e52173cff294fd316f8fbfe283d28148cdf9bc83922e2ce897b922b148a
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/af/84/3962a6af7b4ab336e951b7877dcfb758cf94548bb1771e0679
Successfully built absl-py
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Train the model per the settings specified for ResNext 29_4x64d in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/resnext.yaml \
    model.resnext.cardinality 4 \
    train.batch_size 128 \
    dataset.name CIFAR10_RA_2_5 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00 \
    scheduler.epochs 400

# Number of epochs should be 300!

[2020-06-28 20:42:20] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_2_5
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnext
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck


In [13]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
#!python train.py --config configs/cifar/resnet.yaml \
!python train.py --config /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00/checkpoint_00400.pth \
    model.resnext.cardinality 4 \
    test.batch_size 128 \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00_resume400_50 \
    scheduler.epochs 50

#### Set LEARNING RATE based on ending LR
#    train.resume True \

[2020-07-01 13:12:32] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnext
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_t

In [15]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset - without training on unaugmented data
# write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00/test_results_0400_cifar10


[2020-07-02 17:09:21] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:35<00:00,  2.21it/s]
[2020-07-02 17:09:58] __main__ INFO: Elapsed 35.78
[2020-07-02 17:09:58] __main__ INFO: Loss 0.3952 Accuracy 0.8857


In [16]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset - using model refined on 50 epochs
# write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00_resume400_50/test_results_0050_cifar10

[2020-07-02 17:11:20] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:35<00:00,  2.20it/s]
[2020-07-02 17:11:57] __main__ INFO: Elapsed 35.90
[2020-07-02 17:11:57] __main__ INFO: Loss 0.2333 Accuracy 0.9354


In [17]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# write the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/resnext.yaml \
    model.resnext.cardinality 4 \
    test.batch_size 128 \
    dataset.name CIFAR101 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00/checkpoint_00400.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00/test_results_0400_CIFAR101

[2020-07-02 17:17:48] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:07<00:00,  2.15it/s]
[2020-07-02 17:17:56] __main__ INFO: Elapsed 7.43
[2020-07-02 17:17:56] __main__ INFO: Loss 0.7954 Accuracy 0.7755


In [18]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset - using model refined on 50 epochs
# write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00_resume400_50/test_results_0050_cifar101

[2020-07-02 17:18:50] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:07<00:00,  2.16it/s]
[2020-07-02 17:18:58] __main__ INFO: Elapsed 7.39
[2020-07-02 17:18:58] __main__ INFO: Loss 0.5154 Accuracy 0.8455


In [19]:
import pandas as pd
a = pd.Series(['resnext_29_4x64d_ra_2_5', 400, 'cifar10', 0.3952, 0.8857]) #Loss 0.3952 Accuracy 0.8857
b = pd.Series(['resnext_29_4x64d_ra_2_5', 400, 'cifar10.1', 0.7954, 0.7755]) #0.7954 Accuracy 0.7755

c = pd.Series(['resnext_29_4x64d_ra_2_5_refined400', 50, 'cifar10', 0.2333, 0.9354]) #0.2333 Accuracy 0.9354
d = pd.Series(['resnext_29_4x64d_ra_2_5_refined400', 50, 'cifar10.1', 0.5154, 0.8455]) #0.5154 Accuracy 0.8455

df_results = pd.concat([a,b,c,d], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy_300epochs'] = df_results.apply((lambda row: 96.4 if row[2] == 'cifar10' else 89.6), axis=1)
df_results['Original_CI_300epochs'] = df_results.apply((lambda row: (96.0, 96.7) if row[2] == 'cifar10' else (88.2, 90.9)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_2_5/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy_300epochs,Original_CI_300epochs
0,resnext_29_4x64d_ra_2_5,400,cifar10,0.3952,0.8857,96.4,"(96.0, 96.7)"
1,resnext_29_4x64d_ra_2_5,400,cifar10.1,0.7954,0.7755,89.6,"(88.2, 90.9)"
2,resnext_29_4x64d_ra_2_5_refined400,50,cifar10,0.2333,0.9354,96.4,"(96.0, 96.7)"
3,resnext_29_4x64d_ra_2_5_refined400,50,cifar10.1,0.5154,0.8455,89.6,"(88.2, 90.9)"


In [3]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['resnext_29_4x64d', 'resnext_29_4x64d', 'resnext_29_4x64d', 'resnext_29_4x64d'],
           'Testset': ['cifar10', 'cifar10', 'cifar10', 'cifar10.1'],
           'Epoch': [100, 200, 300, 300],
           'Loss': [0.6746, 0.2311, 0.1517, 0.3742],
           'Accuracy': [0.8019, 0.9321, 0.9535, 0.8905],
           'Original_Accuracy': [96.4, 96.4, 96.4, 89.6],
           'Original_CI': [(96.0, 96.7), (96.0, 96.7), (96.0, 96.7), (88.2, 90.9)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])


df.to_csv('/home/ec2-user/SageMaker/experiments/resnext_29_4x64d/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnext_29_4x64d,cifar10,100,0.6746,0.8019,96.4,"(96.0, 96.7)"
1,resnext_29_4x64d,cifar10,200,0.2311,0.9321,96.4,"(96.0, 96.7)"
2,resnext_29_4x64d,cifar10,300,0.1517,0.9535,96.4,"(96.0, 96.7)"
3,resnext_29_4x64d,cifar10.1,300,0.3742,0.8905,89.6,"(88.2, 90.9)"


In [60]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/resnext_29_4x64d/exp00/test_results_0300/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

['preds', 'probs', 'labels', 'loss', 'acc']


array([[-1.2041907 , -1.8999833 , -0.24285015, ..., -1.5008752 ,
        -1.8426697 , -2.8560946 ],
       [ 0.5460079 ,  2.220384  , -1.9393705 , ..., -2.6070693 ,
        11.327686  , -1.2085156 ],
       [-1.3446747 ,  2.1730833 , -1.1615647 , ..., -2.2299995 ,
        10.984515  , -0.75660706],
       ...,
       [-2.4790986 , -1.3337001 ,  0.61669415, ..., -0.83421385,
        -1.8529658 , -1.7280097 ],
       [-0.90489024,  9.350766  ,  1.0618937 , ..., -2.3210623 ,
        -0.9061641 , -1.8115013 ],
       [-1.4560711 , -1.0518838 , -1.4613396 , ..., 12.668192  ,
        -2.1191459 , -0.8881919 ]], dtype=float32)

In [54]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/'
path = '/home/ec2-user/SageMaker/experiments/'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)

In [43]:
os.getcwd()


'/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass'

## Everything Below Is In-Work

#### SageMaker Way

In [2]:
# S3 Variable Definitions 
bucket='sagemaker-may29'
prefix = '/sagemaker-may29/sagemaker/cifar102/data/cifar10_10k'
model_path = '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/'

role = sagemaker.get_execution_role()
#bucket = sagemaker.Session().default_bucket(
#sagemaker_session = sagemaker.Session()
bucket='sagemaker-may29'
sagemaker_session = sagemaker.Session(default_bucket=bucket)


# Set S3 dataset path 
#inputs = 's3://' + bucket + '/sagemaker/{}'.format(args.train_data)
inputs = 's3://' + bucket + '/sagemaker-may29/sagemaker/cifar102/data/cifar10_10k/cifar10_10k_test'
inputs = 's3://' + bucket + '/sagemaker-may29/sagemaker/cifar102/data/cifar10_10k/'

In [3]:
model_path = '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/'
container_data_dir = '/opt/ml/input/data/training'
container_model_dir = '/opt/ml/model'

parameters = {
    'config': 'resnext.yaml',
    'resnext.depth': 29,
    'train.batch_size': 128,
    'train.base_lr': 0.1,
    #'data_dir': container_data_dir,
    #'dataset.dataset_dir': container_data_dir
    #'output_dir': container_model_dir,
    #'num_train_epochs': 3,
    #'per_gpu_train_batch_size': 64,
    #'per_gpu_eval_batch_size': 64,
    #'save_steps': 150,
    #'logging_steps': 150
} 

In [4]:
# Create an instance of the PyTorch class that enables the model script to run as a 
# training job on the SageMaker distributed, managed training infrastructure
estimator = PyTorch(entry_point= model_path + 'train.py',
                    role=role,
                    framework_version='1.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters=parameters)

In [5]:
# Train the model
#inputs = 's3://' + bucket + '/sagemaker-may29/sagemaker/cifar102/data/cifar10_10k/cifar10_10k_test'
inputs = 's3://' + bucket + '/sagemaker-may29/sagemaker/cifar102/data/cifar10_10k/'
#estimator.fit({'training': inputs})
estimator.fit()

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-06-08 14:49:34 Starting - Starting the training job...
2020-06-08 14:49:36 Starting - Launching requested ML instances.........
2020-06-08 14:51:06 Starting - Preparing the instances for training......
2020-06-08 14:52:21 Downloading - Downloading input data...
2020-06-08 14:53:01 Training - Downloading the training image...........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-06-08 14:54:49,375 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-06-08 14:54:49,397 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-06-08 14:54:49,403 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-06-08 14:54:49,684 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020-06-08 14:54:49,684 sagemaker-containers INFO     Generating setup.cfg
2020-06-08 14:5

UnexpectedStatusException: Error for Training job pytorch-training-2020-06-08-14-49-34-014: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python train.py --config resnext.yaml --resnext.depth 29 --train.base_lr 0.1 --train.batch_size 128"
Traceback (most recent call last):
  File "train.py", line 7, in <module>
    import apex
ModuleNotFoundError: No module named 'apex'

In [8]:
os.getcwd()

'/home/ec2-user/SageMaker'

### Image Augmentation 
https://imgaug.readthedocs.io/en/latest/source/overview/collections.html

In [5]:
!pip install imgaug

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import imgaug.augmenters as iaa
aug = iaa.RandAugment(n=2, m=(0, 9))  
  # n is the number of transformations to apply per image
  # m is magnitude -> specifying a tuple will randomly select values between the min and max (max is 30)

In [26]:
import boto3 
from PIL import Image

cifar10 = "sagemaker/cifar-10/cifar/"
#cifar10_test_rec = "s3://sagemaker-may29/sagemaker/cifar-10/cifar/test.rec"

def download_all_objects_in_folder(b, p):
    s3_resource = boto3.resource('s3')
    my_bucket = s3_resource.Bucket(b)
    objects = my_bucket.objects.filter(Prefix=p)
    for obj in objects:
        path, filename = os.path.split(obj.key)
        my_bucket.download_file(obj.key, filename)    

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Download the unzipped data from the cifar10 folder
download_all_objects_in_folder(bucket, cifar10)



test.rec: data


### RandAugment 
https://arxiv.org/abs/1909.13719
https://pypi.org/project/randaugment/

In [2]:
!pip install randaugment

  Using cached https://files.pythonhosted.org/packages/fb/ea/e24549f459800dc3bed21cd4e9c0d49d5b8deed65214b2444bd3e5a49f30/randaugment-1.0.2-py3-none-any.whl
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from randaugment import RandAugment, ImageNetPolicy
data = ImageFolder('/home/ec2-user/SageMaker/w210-capstone/data/cifar10/subset300', transform=transforms.Compose(
                        [
                            transforms.RandomCrop(32, padding=4, fill=128), # fill parameter needs torchvision installed from source
                            transforms.RandomHorizontalFlip(), 
                            RandAugment(),
                            #ImageNetPolicy(),
                            transforms.ToTensor(), 
                            Cutout(size=16), # (https://github.com/uoguelph-mlrg/Cutout/blob/master/util/cutout.py)
                            transforms.Normalize(...)
                        ])
)

NameError: name 'ImageFolder' is not defined

### Deep Augment
https://blog.insightdatascience.com/automl-for-data-augmentation-e87cf692c366
https://colab.research.google.com/drive/1KCAv2i_F3E3m_PKh56nbbZY8WnaASvgl#scrollTo=SuhR6Q3AMFy3

In [5]:
!pip install deepaugment

    100% |████████████████████████████████| 51kB 4.5MB/s eta 0:00:01
    100% |████████████████████████████████| 8.9MB 5.1MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 34.7MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 33.4MB/s ta 0:00:01
    100% |████████████████████████████████| 13.9MB 3.5MB/s eta 0:00:01
    100% |████████████████████████████████| 34.2MB 1.5MB/s eta 0:00:01
    100% |████████████████████████████████| 645kB 34.7MB/s ta 0:00:01
    100% |████████████████████████████████| 83.1MB 574kB/s eta 0:00:01    62% |███████████████████▉            | 51.6MB 52.9MB/s eta 0:00:01    76% |████████████████████████▋       | 63.8MB 62.9MB/s eta 0:00:01
    100% |████████████████████████████████| 12.9MB 4.0MB/s eta 0:00:01
    100% |████████████████████████████████| 573kB 31.6MB/s ta 0:00:01
    100% |████████████████████████████████| 317kB 37.5MB/s ta 0:00:01
    100% |████████████████████████████████| 1.8MB 18.5MB/s ta 0:00:01
    100% |█████████

In [8]:
from deepaugment.deepaugment import DeepAugment

deepaug = DeepAugment('/home/ec2-user/SageMaker/w210-capstone/data/cifar10/subset300', 
                      '/home/ec2-user/SageMaker/w210-capstone/data/cifar10/trainLabels.csv')

best_policies = deepaug.optimize()

TypeError: __new__() got an unexpected keyword argument 'serialized_options'

In [6]:
!ls /home/ec2-user/SageMaker/w210-capstone/data/cifar10/subset300

/home/ec2-user/SageMaker


In [1]:
# Must be conda_pytroch_p36 notebook
import argparse
import numpy as np
import os
import sagemaker
import torch
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch 

In [2]:
!pip install -r ./w210-capstone/models/pytorch_imageclass/requirements.txt

# Do we need NVIDIA installed?

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
# Must be conda_pytroch_p36 notebook
!python w210-capstone/models/pytorch_imageclass/train.py --config w210-capstone/models/pytorch_imageclass/configs/cifar/resnext.yaml \
   model.resnext.depth 29 \
   model.resnext.cardinality 4 \
   train.batch_size 128 \
   train.base_lr 0.1 \
   train.output_dir experiments/resnext_29_4x64d/exp00

[2020-06-04 00:41:31] __main__ INFO: device: cpu
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnext
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_ty

## SCRATCH

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Data and model checkpoints directories
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--backend', type=str, default=None,
                        help='backend for distributed training (tcp, gloo on cpu and gloo, nccl on gpu)')

    
    parser.add_argument('--epochs', type=int, default=10)
    parser.add_argument('--batch-size', type=int, default=128)
    parser.add_argument('--learning-rate', type=float, default=0.1)
    
    parser.add_argument('--model_name', dest='model_name', type=str, help='model to train')
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'],
                        help='directory to save trained model to')
    parser.add_argument('--train_data', dest='train_data', type=str, help='dataset for model training') 
    parser.add_argument('--workers', dest='workers', type=int, help='number of V100 worker instances; 1 indicates non-distributed training')

    args = parser.parse_args()
    
    
    
    # Container environment
    env = sagemaker_containers.training_env()
    parser.add_argument('--hosts', type=list, default=env.hosts)
    parser.add_argument('--current-host', type=str, default=env.current_host)
    parser.add_argument('--model-dir', type=str, default=env.model_dir)
    parser.add_argument('--data-dir', type=str,
                        default=env.channel_input_dirs['training'])
    parser.add_argument('--num-gpus', type=int, default=env.num_gpus)

    train(parser.parse_args())

In [9]:
#torch.cuda.get_device_name(0)
echo $CUDA_PATH

SyntaxError: invalid syntax (<ipython-input-9-3319f1f978a5>, line 2)

In [16]:
!pip install -r ./w210-capstone/models/pytorch_/requirements.txt

# Do we need NVIDIA installed?

Could not open requirements file: [Errno 2] No such file or directory: './w210-capstone/models/pytorch/requirements.txt'
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Instance type = local


In [10]:
#!nvidia-smi

In [33]:
# ResNext-29_4x64
!python w210-capstone/models/pytorch/train.py --config w210-capstone/models/pytorch/configs/cifar/resnext.yaml \
   model.resnext.depth 29 \
   model.resnext.cardinality 4 \
   train.batch_size 128 \
   train.base_lr 0.1 \
   train.output_dir experiments/resnext_29_4x64d/exp00

# ResNeXt-29 4x64d with a single GPU, batch size 32 and initial learning rate 0.025 
# (8 GPUs, batch size 128 and initial learning rate 0.1 in paper).

[2020-06-03 20:54:56] __main__ INFO: device: cpu
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnext
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_ty

In [ ]:

# initialize command line argument parsing
if __name__=='__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument('--epochs', type=int, default=10)
    parser.add_argument('--batch-size', type=int, default=128)
    parser.add_argument('--learning-rate', type=float, default=0.1)
    
    parser.add_argument('--model_name', dest='model_name', type=str, help='model to train')
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'],
                        help='directory to save trained model to')
    parser.add_argument('--train_data', dest='train_data', type=str, help='dataset for model training') 
    parser.add_argument('--workers', dest='workers', type=int, help='number of V100 worker instances; 1 indicates non-distributed training')

    args = parser.parse_args()

    # Check quality of arguments
    valid_args = {'datasets': ['cifar10', 'cifar10_10k', 'cifar10_30k', 'cifar102', 'cifar102_30k'],
                  'model_names': ['wrn', 'shake_shake_32', 'shake_shake_96', 'shake_shake_112', 'pyramid_net']}

    if args.train_data not in valid_args['datasets']:
        parser.error('Invalid train_data parameter')

    if args.model_name not in valid_args['model_names']:
        parser.error('Invalid model_name parameter')
    
    if args.workers < 1:
        parser.error('Invalid number of workers')

    if not args.model_name:
        parser.error('--model_name parameter is required')
    elif not args.train_data:
        parser.error('--train_data parameter is required')
    elif not args.workers:
        parser.error('--workers parameter is required')

    # Set SageMaker session & execution role
    bucket='sagemaker-may29'
    sagemaker_session = sagemaker.Session(default_bucket=bucket)
    role = get_execution_role()


    # Set S3 path for data batches
    inputs = 's3://' + bucket + '/sagemaker/{}'.format(args.train_data)

    # Create the sagemaker estimator
    pytorch_estimator = PyTorch('./w210-capstone/models/pytorch/train.py',
                                train_instance_type='ml.p3.2xlarge',
                                train_instance_count=1,
                                framework_version='1.0.0',
                                hyperparameters = {'epochs': 20, 'batch-size': 64, 'learning-rate': 0.1})

    --config w210-capstone/models/pytorch/configs/cifar/resnext.yaml \
   model.resnext.depth 29 \
   model.resnext.cardinality 4 \
   train.batch_size 128 \
   train.base_lr 0.1 \
   train.output_dir experiments/resnext_29_4x64d/exp00
    
    
    # Train the Model
    pytorch_estimator.fit({'train': 's3://my-data-bucket/path/to/my/training/data',
                           'test': 's3://my-data-bucket/path/to/my/test/data'})


    # After training, save the model to `model_dir`
    with open(os.path.join(args.model_dir, 'model.pth'), 'wb') as f:
        torch.save(model.state_dict(), f)
        